In [1]:
import os
from os.path import join
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

import torch
import torch.nn as nn
import torchvision
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from astra.torch.utils import train_fn
from astra.torch.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt

device = "cuda"

In [2]:
path = "/home/vannsh.jani/brick_kilns/ssl_exp/data"
x_train = torch.load(join(path, "ban_x_train.pt"))
y_train = torch.load(join(path, "ban_y_train.pt"))
x_test = torch.load(join(path, "ban_x_test.pt"))
y_test = torch.load(join(path, "ban_y_test.pt"))

print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)
x_b = torch.cat([x_train, x_test], dim=0).to(device)
y_b = torch.cat([y_train, y_test], dim=0).to(device)

print(x_b.shape, y_b.shape)

torch.Size([19124, 3, 224, 224]) torch.Size([19124]) torch.Size([6375, 3, 224, 224]) torch.Size([6375])
torch.Size([25499, 3, 224, 224]) torch.Size([25499])


In [3]:
x_d = torch.load(join(path, "delhi_test_images_50.pt")).to(device)
y_d = torch.load(join(path, "delhi_test_labels_50.pt"))
print(x_d.shape, y_d.shape)

torch.Size([5013, 3, 224, 224]) torch.Size([5013])


In [4]:
x_d_train = torch.load(join(path, "delhi_5_images.pt")).to(device)
y_d_train = torch.load(join(path, "delhi_5_labels.pt")).long().to(device)
print(x_d_train.shape, y_d_train.shape)

torch.Size([501, 3, 224, 224]) torch.Size([501])


In [5]:
x_b = torch.cat([x_b, x_d_train], dim=0)
y_b = torch.cat([y_b, y_d_train], dim=0)
print(x_b.shape, y_b.shape)

torch.Size([26000, 3, 224, 224]) torch.Size([26000])


In [6]:
# Downstream model
class DownstreamModel(nn.Module):
    def __init__(self,num_classes):
        super().__init__()
        resnet = torchvision.models.efficientnet_b0(weights=None)
        self.resnet = nn.Sequential(*list(resnet.children())[:-1])
        self.last_layer = list(resnet.children())[-1]
        # print(dict(self.resnet.named_parameters()).keys())
        self.resnet.load_state_dict({k.replace("resnet.", ""): v for k, v in torch.load("/home/vannsh.jani/brick_kilns/ssl_exp/dino/dino_bd_75_delhi_test_train5_final_table.pth").items() if k not in ["fc.weight", "fc.bias"]}) # load different weights
        self.last_layer[1] = nn.Linear(1280, num_classes)

    def forward(self, x):
        x = self.resnet(x).squeeze(-2, -1)
        # print(x.shape)
        x = self.last_layer(x)
        return x

In [7]:
model = DownstreamModel(2).to(device)
model(torch.rand(2, 3, 224, 224).to(device)).shape

torch.Size([2, 2])

In [8]:
lr = 1e-5
epochs = 10
loss_fn = nn.CrossEntropyLoss()

train_losses = []
test_losses = []
for epoch in range(epochs):
    torch.cuda.empty_cache()
    print("Epoch:", epoch)
    model.train()
    iter_losses, epoch_losses = train_fn(model, loss_fn, x_b, y_b, lr=lr, epochs=1, batch_size=64)
    
    torch.cuda.empty_cache()
    
    model.eval()
    
    with torch.no_grad():
        local_losses = []
        for i in range(0, len(x_b), 64):
            y_pred = model(x_b[i:i+64])
            loss = loss_fn(y_pred, y_b[i:i+64].long())
            local_losses.append(loss.item())

        train_losses.append(sum(local_losses)/len(local_losses))
    
    torch.cuda.empty_cache()
    
    with torch.no_grad():
        local_losses = []
        local_y_pres = []
        for i in range(0, len(x_d), 128):
            y_pred = model(x_d[i:i+128]).cpu()
            loss = loss_fn(y_pred, y_d[i:i+128].long())
            local_losses.append(loss.item())
            local_y_pres.append(y_pred.argmax(dim=1))
            
        test_loss = sum(local_losses)/len(local_losses)
        test_losses.append(test_loss)
        y_pred = torch.cat(local_y_pres, dim=0)
        print("Accuracy:", accuracy_score(y_d, y_pred))
        print("Precision:", precision_score(y_d, y_pred))
        print("Recall:", recall_score(y_d, y_pred))
        print("F1 Score:", f1_score(y_d, y_pred))
        print("Train Loss:", train_losses[-1])
        print("Test Loss:", test_losses[-1])
        
    torch.cuda.empty_cache()

Epoch: 0


Loss: 0.42858264: 100%|██████████| 1/1 [00:24<00:00, 24.88s/it]


Accuracy: tensor(0.9611)
Precision: tensor(0.6756)
Recall: tensor(0.9312)
F1 Score: tensor(0.7831)
Train Loss: 0.18599057494129537
Test Loss: 0.2615760453045368
Epoch: 1


Loss: 0.11387730: 100%|██████████| 1/1 [00:24<00:00, 24.90s/it]


Accuracy: tensor(0.9601)
Precision: tensor(0.6564)
Recall: tensor(0.9421)
F1 Score: tensor(0.7738)
Train Loss: 0.050231716977729844
Test Loss: 0.146105345338583
Epoch: 2


...

Accuracy: tensor(0.9561)
Precision: tensor(0.6161)
Recall: tensor(0.9413)
F1 Score: tensor(0.7448)
Train Loss: 0.018634327838777425
Test Loss: 0.19133889134973286
Epoch: 8


Loss: 0.01713382: 100%|██████████| 1/1 [00:24<00:00, 24.90s/it]


Accuracy: tensor(0.9563)
Precision: tensor(0.6180)
Recall: tensor(0.9415)
F1 Score: tensor(0.7462)
Train Loss: 0.01800967013603248
Test Loss: 0.19521590727381408
Epoch: 9


Loss: 0.01629883: 100%|██████████| 1/1 [00:24<00:00, 24.91s/it]


Accuracy: tensor(0.9579)
Precision: tensor(0.6372)
Recall: tensor(0.9379)
F1 Score: tensor(0.7589)
Train Loss: 0.01703857325608968
Test Loss: 0.1934986597392708
